# Comparing The Effectiveness Of RAG Between Models


### Dependencies

In [2]:
!pip install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu128 --quiet
!pip install sentence-transformers faiss-cpu transformers datasets transformers jupyterlab_widgets pandas accelerate numpy hf_xet --quiet

In [3]:
import os
import sys
#------------------------------------------------------------------------------------------------
os.environ["HF_HOME"] = "D:/AI_Models" #-> only for my computer delete on others!!!
#------------------------------------------------------------------------------------------------
import torch
import numpy as np
from numpy.linalg import norm
import faiss
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from accelerate import Accelerator

d:\karl\Large-Language-Modell-Project_\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Test if CUDA is available and set the accelerator

In [5]:

print(f"CUDA verfügbar: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    accelerator = Accelerator()

    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Anzahl GPUs: {torch.cuda.device_count()}")
    print(f"CUDA Version (PyTorch): {torch.version.cuda}")
    print(f"Accelerator device: {accelerator.device}")
else:
    accelerator = Accelerator(cpu=True) 


CUDA verfügbar: True
GPU Name: NVIDIA GeForce RTX 5060 Ti
Anzahl GPUs: 1
CUDA Version (PyTorch): 12.8
Accelerator device: cuda


### Import the models

In [6]:
"""
model_name_mistral = "mistralai/Mistral-7B-v0.1"
model_mistral = AutoModelForCausalLM.from_pretrained(
    model_name_mistral,
    dtype="auto"
    )
model_mistral = accelerator.prepare(model_mistral)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name_mistral)

model_name_deepseek = "deepseek-ai/deepseek-llm-7b-base"
model_deepseek = AutoModelForCausalLM.from_pretrained(
    model_name_deepseek,
    dtype="auto"
    )
model_deepseek = accelerator.prepare(model_deepseek)
tokenizer_deepseek = AutoTokenizer.from_pretrained(model_name_deepseek)

model_name_llama = "meta-llama/Llama-2-7b"
model_llama = AutoModelForCausalLM.from_pretrained(
    model_name_llama,
    dtype="auto"
    )
model_llama = accelerator.prepare(model_llama)
tokenizer_llama = AutoTokenizer.from_pretrained(model_name_llama)
"""

#"Qwen/Qwen2.5-0.5B" -> "Qwen/Qwen2-7B"
model_name_qwen = "Qwen/Qwen2-7B"
model_qwen = AutoModelForCausalLM.from_pretrained(
    model_name_qwen,
    dtype="auto"
)
model_qwen = accelerator.prepare(model_qwen) 
tokenizer_qwen = AutoTokenizer.from_pretrained(model_name_qwen)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.61it/s]


### Import the dataset

In [7]:
"""
trivia_qa = datasets.load_dataset("mandarjoshi/trivia_qa", "rc", split="train")

documents_trivia_qa = [item["search_results"]["context"][0] for item in trivia_qa if item["search_results"]["context"]]
len(documents_trivia_qa)
"""

rag_dataset_1200 = datasets.load_dataset("neural-bridge/rag-dataset-1200", split="train")

documents_rag_1200 = [item["context"] for item in rag_dataset_1200]
len(documents_rag_1200)


960

In [8]:
documents = documents_rag_1200

def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))

len(chunks)

8732

In [9]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2", device=accelerator.device)

embeddings = embed_model.encode(
    chunks,
    show_progress_bar=True
)

embeddings = np.array(embeddings)
embeddings.shape


Batches: 100%|██████████| 273/273 [00:07<00:00, 38.27it/s]


(8732, 384)

We use the FlatL2 index to measure the similarity of the embeddings. FlatL2 ueses the euclidian distance for that.

In [10]:
print(type(embeddings))
print(len(embeddings))  
if len(embeddings) > 0:
    print(np.array(embeddings).shape)

dimension = embeddings.shape[1]


index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

index.ntotal

<class 'numpy.ndarray'>
8732
(8732, 384)


8732

In [11]:
def retrieve_context(question, k=10):
    q_emb = embed_model.encode([question])
    distances, indices = index.search(q_emb, len(chunks))  
    selected_chunks = [chunks[i] for i in indices[0] if len(chunks[i].strip()) > 50]
    return "\n".join(selected_chunks[:k])  

Prompt similar the the one from the Ragas paper

In [12]:
def build_rag_prompt(context, question):
    return f"""
Answer the question using ONLY the information from the context.
Return ONLY complete sentences.
Do NOT include explanations, commentary, or unrelated text.
If the answer is not explicitly stated, reply with: I don't know.

Context:
{context}

Question:
{question}

Answer:
"""


In [ ]:
def answer_without_context(model, tokenizer, prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(accelerator.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True, #-> evtl do_sample auf false
            temperature=0.01,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    generated = output_ids[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()


In [ ]:
def answer_with_context(model, tokenizer, question, max_tokens=400, k=10):
    context = retrieve_context(question, k=k)
    prompt = build_rag_prompt(context, question)
    inputs = tokenizer(prompt, return_tensors="pt").to(accelerator.device)
    prompt_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True, #-> evtl do_sample auf false
            temperature=0.01,    
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )

    generated = output_ids[0][prompt_len:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()


### The three scores mentioned in the Ragas paper
In this section we implement all three scores mentioned in the Ragas paper.
We will use these to evaluate the models rag performance

Faithfulness Score: $$F = \frac{|F|}{|V|}$$

In [ ]:
def build_faithfullness_prompt(statements, context):
    prompt = f"""
Context:
{context}

Consider the given context and following statements, the determine whether they are supported by the information presente in the context. 
Provide a brief explanation for each statement before arriving at the verdict (Yes/No). 
Provide a final verdict for each statement in order at the end in the given format. 
Do not deviate from the specified format.
statement: [statement 1]
...
statement: [statement n]

Now evaluate the following statements:
"""
    for s in statements:
        prompt += f'Statement: "{s}"\nAnswer: '

    return prompt


In [ ]:
def extract_statements(question: str, answer: str, model, tokenizer, max_tokens=150):
    #Extracts statements out of the model generated answer 
    """
    Splits a model-generated answer into complete factual statements.
    Each statement will be one line. Partial words or fragments are avoided.
    """

    prompt = f"""Given a question and answer, create one or more statements from each sentence in the given answer.
question: {question}
answer: {answer}"""


    inputs = tokenizer(prompt, return_tensors="pt")

    
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,      
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id
        )

    
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    
    statements = [line.strip() for line in output_text.split("\n") if len(line.strip()) > 3]

    return statements


In [ ]:
def evaluate_faithfulness(statements, context, model, tokenizer):
    if len(statements) == 0:
        return 0.0

    prompt = build_faithfullness_prompt(statements, context)
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False,      
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)


    answer_lines = [line.split("Answer:")[-1].strip() for line in output.split("\n") if "Answer:" in line]

    supported = sum(1 for line in answer_lines if line.lower().startswith("yes"))

    return supported / len(statements)


Answer Relevance Score: $$AR = \frac{1}{n} \sum\limits^n_{i=1} sim(q,q_i)$$

In [ ]:
def eval_similarity(q: str, q_answer: str) -> float:
    emb = embed_model.encode([q, q_answer])
    sim = np.dot(emb[0], emb[1]) / (norm(emb[0]) * norm(emb[1]))
    return (sim + 1) / 2

In [ ]:
def generate_questions_from_answer(answer: str, model, tokenizer, max_tokens=150):
    prompt = f"""
Generate up to 3 questions that could be answered by the following answer.
Return one question per line.

Answer:
{answer}
"""
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return [q.strip() for q in output.split("\n") if q.strip().endswith("?")]


In [ ]:
def calculate_answer_relevance_score(question: str, generated_questions: list):
    if len(generated_questions) == 0:
        return 0.0

    score = 0
    for q in generated_questions:
        score += eval_similarity(question, q)

    return score / len(generated_questions)


In [ ]:
def calculate_answer_relevance_direct(question: str, answer: str):
    """
    Misst die semantische Ähnlichkeit zwischen Frage und Antwort direkt
    und skaliert sie auf [0,1].
    """
    if not answer or answer.lower() in ["i don't know", "unknown"]:
        return 0.0


    emb = embed_model.encode([question, answer])
    
  
    sim = np.dot(emb[0], emb[1]) / (norm(emb[0]) * norm(emb[1]))
    
 
    return (sim + 1) / 2


Context Relevance Score: $$CR = \frac{\text{number of extracted sentences}}{\text{total number of senctences in }c(q)}$$

In [ ]:
def build_context_relevance_prompt():
    return f"""Please extract relevant sentences from the provided context that can potentially help ansewr the following question.
If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phares \"Insufficient Information\".
While extracting candidate sentences you're not allowed to make any changes to senctences from the given context."""

In [ ]:
def evaluate_rag_answer(question, answer, context, model, tokenizer):
 
    if answer.lower() in ["i don't know", ""]:
        return {
            "faithfulness": 0.0,
            "answer_relevance": 0.0,
            "hallucination": True
        }

    statements = extract_statements(answer, model, tokenizer)[:10]

    


    if not statements:
        faithfulness = 0.0
    else:
        faithfulness = evaluate_faithfulness(statements, context, model, tokenizer)


    answer_relevance = calculate_answer_relevance_direct(question, answer)


    return {
        "faithfulness": faithfulness,
        "answer_relevance": answer_relevance,
        "hallucination": faithfulness < 1.0
    }


### RAG analysis

In [ ]:
def analyse():
    results = []
    for i in range(2):  
        question = rag_dataset_1200[i]["question"]
        context = rag_dataset_1200[i]["context"]
        answer = answer_with_context(model_qwen, tokenizer_qwen, question, max_tokens=50, k=5)

        scores = evaluate_rag_answer(
            question, answer, context, model_qwen, tokenizer_qwen
        )

        results.append({
            "question": question,
            "answer": answer,
            **scores
        })
    return results


In [ ]:
"""
question = "Who were the three stars in the NHL game between Buffalo Sabres and Edmonton Oilers?"


context = retrieve_context(question, k=5)

print("RETRIEVED CONTEXT:")
print(context)
print("\n---\n")



answer = answer_with_context(
    model=model_qwen, 
    tokenizer=tokenizer_qwen, 
    question=question,  
    max_tokens=200,
    k=5
)

print("ANSWER:")
print(answer)
print("\n---\n")


scores = evaluate_rag_answer(
    question=question,
    answer=answer,
    context=context,
    model=model_qwen,
    tokenizer=tokenizer_qwen
)

print("EVALUATION SCORES:")
print(scores)
"""

'\nquestion = "Who were the three stars in the NHL game between Buffalo Sabres and Edmonton Oilers?"\n\n\ncontext = retrieve_context(question, k=5)\n\nprint("RETRIEVED CONTEXT:")\nprint(context)\nprint("\n---\n")\n\n\n\nanswer = answer_with_context(\n    model=model_qwen, \n    tokenizer=tokenizer_qwen, \n    question=question,  \n    max_tokens=200,\n    k=5\n)\n\nprint("ANSWER:")\nprint(answer)\nprint("\n---\n")\n\n\nscores = evaluate_rag_answer(\n    question=question,\n    answer=answer,\n    context=context,\n    model=model_qwen,\n    tokenizer=tokenizer_qwen\n)\n\nprint("EVALUATION SCORES:")\nprint(scores)\n'